# ID3 (Iterative Dichotomiser 3) Invented by J.Ross Quinlan in 1975.

The following snippet shows the algorithm from pg 56 of Machine Learning by Tom M. Mitchell

![title](./../ref/ID3_algorithm.PNG)

In [4]:
DATA_CSV_FILE_PATH = './../data/tennis.csv'
TARGET_COLUMN = 'play'
TARGET_POSITIVE_VALUE = 'yes'

In [5]:
import random

import numpy as np
import pandas as pd

from __future__ import print_function
from __future__ import division

In [6]:
data = pd.read_csv(DATA_CSV_FILE_PATH) 
attributes_data = data.drop(TARGET_COLUMN,axis=1)
target = data[TARGET_COLUMN]

In [7]:
attributes_data[:3]

,outlook,temp,humidity,windy
0,sunny,hot,high,False
1,sunny,hot,high,True
2,overcast,hot,high,False


In [8]:
target[:3]

0     no
1     no
2    yes
Name: play, dtype: object

## This algorithms works in the following setting
1. The possible values of target are 2.
2. All attribute have discrete values. Target values are also discrete values.

### Measuring partition effectiveness / purity

The following image taken from https://sebastianraschka.com/faq/docs/decision-tree-binary.html shows how different impurity measures change when p -> proportion of positive example changes

![title](./../ref/impurity measures.png)

Entropy is calculated as

![title](./../ref/Entropy.jpg)

In [9]:
'''
p is proportion of positive examples.
'''
def get_positive_fraction(target):
    return len(target[target==TARGET_POSITIVE_VALUE])/len(target)

def gini(p):
    return (p)*(1 - (p)) + (1 - p)*(1 - (1-p))

def entropy(p):
    if p==0 or p==1:
        return 0.0
    return - p*np.log2(p) - (1 - p)*np.log2((1 - p))

def classification_error(p):
    return 1 - np.max([p, 1 - p])

In [10]:
p = get_positive_fraction(target)
print('gini : {} | entropy : {} | classification_error : {}'.format(gini(p), entropy(p), classification_error(p)))

gini : 0.459183673469 | entropy : 0.940285958671 | classification_error : 0.357142857143


In [11]:
def most_common(target):
    return target.value_counts().idxmax()

In [12]:
most_common(target)

'yes'

### Information gain of the attribute

Information Gain is defined as 
![title](./../ref/InformationGain.jpg)

In [13]:
def get_subsets(data,attribute,target):
    subset_list = []
    attribute_values = data[attribute].unique()
    for value in attribute_values:
        value_filter = data[attribute] == value
        subset = (data[value_filter],target[value_filter])
        subset_list.append(subset)
    return subset_list

In [14]:
get_subsets(attributes_data,attributes_data.columns[0],target)[1][0]

,outlook,temp,humidity,windy
2,overcast,hot,high,False
6,overcast,cool,normal,True
11,overcast,mild,high,True
12,overcast,hot,normal,False


In [15]:
def information_gain(data,attribute,target):
    positive_label_fraction = get_positive_fraction(target)
    attribute_entropy = entropy(positive_label_fraction)    
    
    subset_list = get_subsets(data,attribute,target)
    sum_subset_entropy = 0.0
    for subset in subset_list:
        subset_data,subset_target = subset
        sum_subset_entropy = sum_subset_entropy + (len(subset_target)/len(target)) * entropy(get_positive_fraction(subset_target))
        
    inf_gain = attribute_entropy - sum_subset_entropy
    return inf_gain

In [16]:
for attribute in attributes_data.columns:
    print('information gain for {} is {}'.format(attribute,information_gain(attributes_data,attribute,target)))

information gain for outlook is 0.246749819774
information gain for temp is 0.029222565659
information gain for humidity is 0.151835501362
information gain for windy is 0.0481270304083


In [17]:
def get_best_attribute(data, attributes, target):

    best = attributes[0]
    max_gain = 0.0;

    for attribute in attributes:
        gain = information_gain(data,attribute,target) 
        if gain > max_gain:
            max_gain = gain
            best_attribute = attribute

    return best_attribute,max_gain

In [18]:
get_best_attribute(attributes_data,attributes_data.columns,target)

('outlook', 0.24674981977443933)

### Build Tree
Underlying data structure for ID3 tree is nested dictionaries

In [19]:
def build_ID3_tree(data,target,attributes):
    
    tree = {}
    if len(target.unique()) == 1:
        label = target.unique()[0]
        return label
    
    if len(attributes) == 1 or len(data) == 0:
        return most_common(target)
    
    best_attribute,information_gain = get_best_attribute(data,attributes,target)
    tree = {best_attribute:{}}
    
    subset_list = get_subsets(data,best_attribute,target)
    
    for subset,subset_target in subset_list:
        subset_value = subset[best_attribute].unique()[0]
        subset = subset.drop(best_attribute,axis=1)
        subtree = build_ID3_tree(subset,subset_target,subset.columns) 
        tree[best_attribute][subset_value] = subtree
        
    return tree

In [20]:
# All positive examples
tree = build_ID3_tree(attributes_data,target,attributes_data.columns)

In [21]:
tree

{'outlook': {'overcast': 'yes',
  'rainy': {'windy': {False: 'yes', True: 'no'}},
  'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}

### Test on unseen data

In [22]:
attributes_data[:3]

,outlook,temp,humidity,windy
0,sunny,hot,high,False
1,sunny,hot,high,True
2,overcast,hot,high,False


In [23]:
test_size = 10

unseen_attributes_data = pd.DataFrame({attribute: [random.choice(attributes_data[attribute].unique()) for item in range(test_size)] 
                                       for attribute in attributes_data})
unseen_attributes_data[:5]

,humidity,outlook,temp,windy
0,normal,rainy,cool,True
1,normal,overcast,hot,False
2,normal,overcast,hot,False
3,high,overcast,hot,False
4,high,overcast,cool,True


In [24]:
def classify(tree,rec_dict):
    for attribute,sub_tree in tree.items():
        attribute_value = rec_dict[attribute]
        value_tree = sub_tree[attribute_value]
        if isinstance(value_tree,dict):
            return classify(value_tree,rec_dict)
        else:
            return value_tree

In [25]:
predictions = [classify(tree,record) for record in unseen_attributes_data.to_dict('records')]
unseen_data = pd.concat([unseen_attributes_data,pd.Series(predictions,name=TARGET_COLUMN)],axis=1)
unseen_data

,humidity,outlook,temp,windy,play
0,normal,rainy,cool,True,no
1,normal,overcast,hot,False,yes
2,normal,overcast,hot,False,yes
3,high,overcast,hot,False,yes
4,high,overcast,cool,True,yes
5,normal,overcast,cool,False,yes
6,normal,sunny,mild,True,yes
7,normal,rainy,cool,False,yes
8,normal,rainy,cool,False,yes
9,high,sunny,hot,False,no
